#  Heuristic Search





#What's a Heuristic?

Heuristic is a way to find an approximate solution faster than classic methods can find an exact solution. Heuristic functions take a look at search algorithms. At each step, it ranks alternatives, making decisions based on available informations. Even though they are effective in a lot of cases, heuristic functions are not guarantee to work for every case. 



#Why do we need heuristics?

In the context of machine learning, classic methods of finding a good solution to the algorithm selection and configuration problem are either:
- A grid search, which becomes very computationally expensive very quickly with the increase of parameter space.
- A random search, which can be faster than a grid search, but with which finding a good solution is completely up to chance.

Heuristic approaches come as alternatives to these options which can (consistently) find reasonably good solutions in a relatively short time. The solutions produced will not be necessarily the best, but are at least better than using a default algorithm/configuration.

#Heuristic Search Techniques in AI

Heuristic Search Techniques can be categorized in two groups: 




## Direct Heuristic Search Techniques in AI

Heuristic Search Techniques of this category search the entiry space to find a solution and use an arbitrary ordering of operations. Their viability are low because they use a lot of time or memory. Breadth First Search (BFS) and Depth First Search (DFS) are examples of this group. 

Other names we can use to classify this group are: Blind Search, Uninformed Search, and Blind Control Strategy.

## Weak Heuristic Search Techniques in AI

Also called as Informed Search, Heuristic Search, and Heuristic Control Strategy, heuristic functions of this group are effective when applied correctly to the right kind of tasks. For this group, we need extra informations, like domain-specific data, to explore, expand and compute preference among child nodes.

Somes examples of heuristic function of this category are: Best-First Search, A* Search, Bidirectional Search, Tabu Search, Beam Search, Simulated Annealing, Hill Climbing and Constraint Satisfaction Problems (CSP).

#Tune-sklearn 

Now bringing to the scikit-learn, it provides us two methods out-of-the-box to address hyperparameter tuning: the Grid Search (GridSearchCV) and Random Search (RandomizedSearchCV), both mentioned previously. But, as mentioned, these are respectivelly really time costly and really unrealiable, since they use "brutal force" to find the right hyperparameter configurations. To overcome that, we can use Tune-sklearn.

Tune-sklearn is a substitute for the Scikit-Learn's model selection module. Using techniques like bayesian optimization, early stopping and distributed execution, it provides a speedup over the Grid Search and Randon Search techniques.

Somes vantages of Tune-sklearn are:

*   It works really well with scikit-learn: both have a very similar interface, and so you don't need to change a lot of lines in a standard scikit-learn script to use Tune-sklearn.
*   It presents Framework support:  tune-it is used primarily for tuning Scikit-Learn models, but it supports other frameworks with Scikit-Learn wrappers, like: Skorch (Pytorch), KerasClassifiers (Keras), and XGBoostClassifiers (XGBoost).
*   Scales up: it helps the Ray Tune, library for distributed hyperparameter tuning, to parallelize cross validation on multiple cores in a transparently and efficiently way.
















#First Steps

At first, we need to install the libraries that we are going to use. In this case, we are going to use the:


*   scikit-optimize
*   tune_sklearnk
*   tune-sklearn "ray[tune]"



In [ ]:
!pip install scikit-optimize
!pip install tune-sklearn "ray[tune]"

     |████████████████████████████████| 102kB 2.3MB/s 
ERROR: Could not find a version that satisfies the requirement tune_sklearnk (from versions: none)
ERROR: No matching distribution found for tune_sklearnk
     |████████████████████████████████| 22.9MB 1.3MB/s 
     |████████████████████████████████| 204kB 31.5MB/s 
     |████████████████████████████████| 81kB 8.1MB/s 
     |████████████████████████████████| 133kB 39.6MB/s 
     |████████████████████████████████| 1.2MB 40.8MB/s 
     |████████████████████████████████| 2.9MB 40.4MB/s 
     |████████████████████████████████| 71kB 7.6MB/s 
     |████████████████████████████████| 71kB 7.2MB/s 
     |████████████████████████████████| 317kB 37.2MB/s 
     |████████████████████████████████| 266kB 42.9MB/s 
     |████████████████████████████████| 153kB 34.0MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 102kB 7.8MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp36-none-any.whl s

#TuneGridSearchCV Example

We are going to start by doing an example of a Tune Grid Search.

First, we are going to do somes imports that we are going to need during the process.

We are going to use a custom "dummy" dataset and SGDClassifier to classify the data. This classifier provides us the partial_fit API, which allows us to stop fitting data for a certain hyperparameter configuration when it is no longer beneficial. If the estimator does not support early stopping, we fall back to a parallel grid search.

In [ ]:
import numpy as np
from tune_sklearn import TuneSearchCV
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier

from tune_sklearn import TuneGridSearchCV

As you can see below, we do the setup in the same way we would do for Scikit-Learn.

In [ ]:
# Set training and validation sets
X, y = make_classification(n_samples=11000, n_features=1000, n_informative=50, 
                           n_redundant=0, n_classes=10, class_sep=2.5)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1000)

# Example parameters to tune from SGDClassifier
parameters = {
   'alpha': [1e-4, 1e-1, 1],
   'epsilon':[0.01, 0.1]
}

Now, we are going to try fitting a model.

Some differences that you will notice:



*   a new *early_stopping* variable: it says when to stop early - it uses the MedianStoppingRule by default, but there is a list of possilities that you can see in [Tune’s documentation](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html);
*   a specification of *max_iters* parameter: it's the maximun number of iterations that a set o Hyperparameters can run; if the process is stopped early, it may have less iteration. 



In [ ]:
tune_search = TuneGridSearchCV(
    SGDClassifier(),
    parameters,
    early_stopping=True,
    max_iters=10
)

import time # Just to compare fit times

start = time.time()
tune_search.fit(X_train, y_train)
end = time.time()
print("Tune Fit Time:", end - start)

pred = tune_search.predict(X_test)
accuracy = np.count_nonzero(np.array(pred) == np.array(y_test)) / len(pred)
print("Tune Accuracy:", accuracy)

Tune Fit Time: 41.78004288673401
Tune Accuracy: 0.878


To do a comparation, below we have an equivalete to the code we run before using GridSearchCV. Pay attention to the results and the difference between the running times.

In [ ]:
from sklearn.model_selection import GridSearchCV
# n_jobs=-1 enables use of all cores like Tune does
sklearn_search = GridSearchCV(
   SGDClassifier(),
   parameters,
   n_jobs=-1
)

start = time.time()
sklearn_search.fit(X_train, y_train)
end = time.time()
print("Sklearn Fit Time:", end - start)
pred = sklearn_search.predict(X_test)
accuracy = np.count_nonzero(np.array(pred) == np.array(y_test)) / len(pred)
print("Sklearn Accuracy:", accuracy)

Sklearn Fit Time: 122.666175365448
Sklearn Accuracy: 0.878


#TuneSearchCV Bayesian Optimization Example

TuneSearchCV is an interface that allow us to sample from distributions of hyperparameters. It also enables to use Bayesian optimization over the distribution with an addition of a few lines.

Bayesian optimization is an approach to optimizing objective functions that take a long time to evaluate. Its strategy is to treat the objective function as a random function and model an a priori probability distribution after it, since the real function is unknown. It has been used in a lot of areas, such as: computer graphics and visual design, architecture configuration in deep learning, automatic algorithm configuration and others.

To run the code below, we need to install the library "scikit-optimize", which we've already done.

In this example, we will continue to use the a "dummy" dataset and SGDClassifier as a classifier.

In [ ]:
from tune_sklearn import TuneSearchCV

# Other imports
import scipy
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier

# Set training and validation sets
X, y = make_classification(n_samples=11000, n_features=1000, n_informative=50, 
                           n_redundant=0, n_classes=10, class_sep=2.5)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1000)

# Example parameter distributions to tune from SGDClassifier
# Note the use of tuples instead if Bayesian optimization is desired
param_dists = {
   'alpha': (1e-4, 1e-1),
   'epsilon': (1e-2, 1e-1)
}

tune_search = TuneSearchCV(SGDClassifier(),
   param_distributions=param_dists,
   n_trials=2,
   early_stopping=True,
   max_iters=10,
   search_optimization="bayesian"
)

tune_search.fit(X_train, y_train)
print(tune_search.best_params_) 

{'alpha': 0.0870109160675465, 'epsilon': 0.061442965886015694}


As you can see, it’s pretty simple to use tune-sklearn and integrate it to an existing code. But, as any heuristic method, it will not suffice for every application, since it doesn't always finds the best solutions.

If you want to know more about tune-sklearn, here are a few links that might be useful:

*   [Tune-sklearn documentation](https://github.com/ray-project/tune-sklearn)

*   [ Scikit-Learn Pipelines with tune-sklearn](https://github.com/ray-project/tune-sklearn/blob/master/examples/sklearn_pipeline.py)

